Automation Final Project
=============

## Part 1:
Resize, rotate, change extention ans save image in another folder.
### Libraries used: 
PIL and os


In [ ]:
#!/usr/bin/env python3

import os
from PIL import Image

path = '/home/student-00-f7e585e91ff6/images/'
new_location = '/opt/icons/'
images = os.listdir(path)
new_size = (128, 128)

for image in images:
        if image != 'changeImages.py':
                with Image.open('{}{}'.format(path, image)) as img:
                        img = img.rotate(90)
                        img = img.resize(new_size)

                        f, e = os.path.splitext(image)
                        out = f + '.jpeg'
                        img = img.convert('RGB').save('{}{}.jpeg'.format(new_location, image))


## Part 2:

    -Use the Python OS module to process a directory of text files
    -Manage information stored in Python dictionaries
    -Use the Python requests module to upload content to a running Web service
    -Understand basic operations for Python requests like GET and POST methods

### Libraries used: 
Requests and os


In [ ]:
#! /usr/bin/env python3

import os
import requests

files = os.listdir('/data/feedback/')

out_dict = []
feedback = {}
lines = []

for file in files:
        if '.txt' in file:
                with open ('/data/feedback/{}'.format(file)) as f:
                        for line in f:
                                lines.append(line)

                        feedback['title'] = lines[0]
                        feedback['name'] = lines[1]
                        feedback['date'] = lines[2]
                        feedback['feedback'] = lines[3]

                        response = requests.post('http://34.71.37.185/feedback', json=feedback)
                        print(response.status_code)

                        lines.clear()


### Second Solution (solved):

In [ ]:
#! /usr/bin/env python3
import os
import requests
feedbacks=[]
floder=(os.listdir("/data/feedback"))
for i in floder:
        with open('/data/feedback/'+i,'r') as file1:
                lines=file1.readlines()
                feedback={"title":lines[0].replace("\n",''),"name":lines[1].replace("\n",''),"date":lines[2].replace("\n",''),"feedback":lines[3].replace("\n",'')}
                feedbacks.append(feedback)
for j in feedbacks:
        respons=requests.post("http://34.71.37.185/feedback/",json=j)


## Part 3:

    -Write a script that summarizes and processes sales data into different categories
    -Generate a PDF using Python
    -Automatically send a PDF by email 


### Generate report:

In [ ]:
#!/usr/bin/env python3
from reportlab.platypus import SimpleDocTemplate
from reportlab.platypus import Paragraph, Spacer, Table, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors


def generate(filename, title, additional_info, table_data):
  styles = getSampleStyleSheet()
  report = SimpleDocTemplate(filename)
  report_title = Paragraph(title, styles["h1"])
  report_info = Paragraph(additional_info, styles["BodyText"])
  table_style = [('GRID', (0,0), (-1,-1), 1, colors.black),
                ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
                ('ALIGN', (0,0), (-1,-1), 'CENTER')]
  report_table = Table(data=table_data, style=table_style, hAlign="LEFT")
  empty_line = Spacer(1,20)
  report.build([report_title, empty_line, report_info, empty_line, report_table])


### Create email

In [ ]:
#!/usr/bin/env python3
import email.message
import mimetypes
import os.path
import smtplib

def generate(sender, recipient, subject, body, attachment_path):
  """Creates an email with an attachement."""
  # Basic Email formatting
  message = email.message.EmailMessage()
  message["From"] = sender
  message["To"] = recipient
  message["Subject"] = subject
  message.set_content(body)

  # Process the attachment and add it to the email
  attachment_filename = os.path.basename(attachment_path)
  mime_type, _ = mimetypes.guess_type(attachment_path)
  mime_type, mime_subtype = mime_type.split('/', 1)

  with open(attachment_path, 'rb') as ap:
    message.add_attachment(ap.read(),
                          maintype=mime_type,
                          subtype=mime_subtype,
                          filename=attachment_filename)

  return message

def send(message):
  """Sends the message to the configured SMTP server."""
  mail_server = smtplib.SMTP('localhost')
  mail_server.send_message(message)
  mail_server.quit()


### Send email:

In [ ]:
#!/usr/bin/env python3
import emails
import os
import reports

table_data=[
  ['Name', 'Amount', 'Value'],
  ['elderberries', 10, 0.45],
  ['figs', 5, 3],
  ['apples', 4, 2.75],
  ['durians', 1, 25],
  ['bananas', 5, 1.99],
  ['cherries', 23, 5.80],
  ['grapes', 13, 2.48],
  ['kiwi', 4, 0.49]]
reports.generate("/tmp/report.pdf", "A Complete Inventory of My Fruit", "Thi$

sender = "automation@example.com"
receiver = "{}@example.com".format(os.environ.get('USER'))
subject = "List of Fruits"
body = "Hi\n\nI'm sending an attachment with all my fruit."

message = emails.generate(sender, receiver, subject, body, "/tmp/report.pdf")
emails.send(message)


### Main:

In [ ]:
#!/usr/bin/env python3

import json
import locale
import sys
from reports import generate as report
from emails import generate as email_generate
from emails import send as email_send


def load_data(filename):
    """Loads the contents of filename as a JSON file."""
    with open(filename) as json_file:
        new_data = json.load(json_file)
        data = sorted(new_data, key=lambda i: i['total_sales'])
    return data


def format_car(car):
    """Given a car dictionary, returns a nicely formatted name."""
    return "{} {} ({})".format(
        car["car_make"], car["car_model"], car["car_year"])


def process_data(data):
    """Analyzes the data, looking for maximums.

  Returns a list of lines that summarize the information.
  """
    locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
    max_revenue = {"revenue": 0}
    sales = {"total_sales": 0}
    best_car = {}
    for item in data:
        # Calculate the revenue generated by this model (price * total_sales)
        # We need to convert the price from "$1234.56" to 1234.56
        item_price = locale.atof(item["price"].strip("$"))
        item_revenue = item["total_sales"] * item_price
        if item_revenue > max_revenue["revenue"]:
            item["revenue"] = item_revenue
            max_revenue = item
        # TODO: also handle max sales
        if item["total_sales"] > sales["total_sales"]:
            sales = item
        # TODO: also handle most popular car_year
        if not item["car"]["car_year"] in best_car.keys():
            best_car[item["car"]["car_year"]] = item["total_sales"]
        else:
            best_car[item["car"]["car_year"]] += item["total_sales"]

        all_values = best_car.values()
        max_value = max(all_values)
        max_key = max(best_car, key=best_car.get)

    summary = [
        "The {} generated the most revenue: ${}".format(
            format_car(max_revenue["car"]), max_revenue["revenue"]),
        "The {} had the most sales: {}".format(sales["car"]["car_model"], sales["total_sales"]),
        "The most popular year was {} with {} sales.".format(max_key, max_value),
    ]

    return summary


def cars_dict_to_table(car_data):
    """Turns the data in car_data into a list of lists."""
    table_data = [["ID", "Car", "Price", "Total Sales"]]
    for item in car_data:
        table_data.append([item["id"], format_car(item["car"]), item["price"], item["total_sales"]])
    return table_data


def main(argv):
    """Process the JSON data and generate a full report out of it."""
    data = load_data("/home/<username>/car_sales.json")
    summary = process_data(data)
    new_summary = '<br/>'.join(summary)
    print(summary)
    # TODO: turn this into a PDF report
    report('/tmp/cars.pdf', "Cars report", new_summary, cars_dict_to_table(data))
    # TODO: send the PDF report as an email attachment
    msg = email_generate("automation@example.com", "<username>@example.com",
                         "Sales summary for last month", new_summary, "/tmp/cars.pdf")
    email_send(msg)


if __name__ == "__main__":
    main(sys.argv)


## Part 4:

    -Write a script that summarizes and processes sales data into different categories
    -Generate a PDF using Python
    -Automatically send a PDF by email
    -Write a script to check the health status of the system 

### changeImage.py

In [ ]:
#!/usr/bin/env python3

import os, sys
from PIL import Image

user = os.getenv('USER') # To get the username from environment variable
image_directory = '/home/{}/supplier-data/images/'.format(user)

for image_name in os.listdir(image_directory):
    if not image_name.startswith('.') and 'tiff' in image_name:
        
        image_path = image_directory + image_name
        path = os.path.splitext(image_path)[0]
        im = Image.open(image_path)
        new_path = '{}.jpeg'.format(path)
        im.convert('RGB').resize((600, 400)).save(new_path, "JPEG")

### supplier_image_upload.py

In [ ]:
#!/usr/bin/env python3
import requests, os

# The URL to upload the images
url = "http://localhost/upload/"
# To get the username from environment variable
USER = os.getenv('USER')
# The directory which contains all the images.
image_directory = '/home/{}/supplier-data/images/'.format(USER)
# Listing all the files in images directory
files = os.listdir(image_directory)

# Parsing through all the images
for image_name in files:
    # Accepting files that has jpeg extension and ignoring hidden files
    if not image_name.startswith('.') and 'jpeg' in image_name:
        # creating absolute path for each image
        image_path = image_directory + image_name
        
        # uploading jpeg files
        with open(image_path, 'rb') as opened:
            r = requests.post(url, files={'file': opened})


### run.py

In [ ]:
#!/usr/bin/env python3

import os, requests, json

def catalog_data(url,description_dir):
    """This function will return a list of dictionaries with all the details like name, weight, description, image_name.
    It will change the weight to integer format.
    """
    fruit={}
    for item in os.listdir(description_dir):
      fruit.clear()
      filename=os.path.join(description_dir,item)
      with open(filename) as f:
        line=f.readlines()
        description=""
        
        for i in range(2,len(line)):
          description=description+line[i].strip('\n').replace(u'\xa0',u'')
        fruit["description"]=description
        fruit["weight"]=int(line[1].strip('\n').strip('lbs'))
        fruit["name"]=line[0].strip('\n')
        fruit["image_name"]=(item.strip('.txt'))+'.jpeg'
        print(fruit)
        
        if url!="":
          response=requests.post(url, json=fruit)
          print(response.request.url)
          print(response.status_code)
    return 0
        
if __name__=='__main__':
    url = 'http://localhost/fruits/'
    user = os.getenv('USER')
    description_directory = '/home/{}/supplier-data/descriptions/'.format(user)
    catalog_data(url,description_directory)

### reports.py

In [ ]:
#!/usr/bin/env python3

from reportlab.platypus import Paragraph, Spacer, Image, SimpleDocTemplate
from reportlab.lib.styles import getSampleStyleSheet

def generate_report(file, title, add_info):
    styles = getSampleStyleSheet()
    report = SimpleDocTemplate(file)
    report_title = Paragraph(title, styles['h1'])
    report_info = Paragraph(add_info, styles['BodyText'])
    empty_line = Spacer(1,20)

    report.build([report_title, empty_line, report_info, empty_line])


### report_email.py

In [ ]:
#!/usr/bin/env python3

import datetime
import os

from run import catalog_data
from reports import generate_report
from emails import generate_email, send_email


def pdf_body(input_for,desc_dir):
    """Generating a summary with two lists, which gives the output name and weight"""
    res = []
    wt = []
    
    for item in os.listdir(desc_dir):
      filename=os.path.join(desc_dir,item)
      with open(filename) as f:
        line=f.readlines()
        weight=line[1].strip('\n')
        name=line[0].strip('\n')
        print(name,weight)
        res.append('name: ' +name)
        wt.append('weight: ' +weight)
        print(res)
        print(wt)
    new_obj = ""  # initializing the object
    
    # Calling values from two lists one by one.
    for i in range(len(res)):
        if res[i] and input_for == 'pdf':
            new_obj += res[i] + '<br />' + wt[i] + '<br />' + '<br />'
    return new_obj

if __name__ == "__main__":
    user = os.getenv('USER')
    description_directory = '/home/{}/supplier-data/descriptions/'.format(user)  # The directory which contains all the files with data in it.
    current_date = datetime.date.today().strftime("%B %d, %Y")  # Creating data in format "May 5, 2020"
    title = 'Processed Update on ' + str(current_date)  # Title for the PDF file with the created date
    generate_report('/tmp/processed.pdf', title, pdf_body('pdf',description_directory))  # calling the report function from custom module
    email_subject = 'Upload Completed - Online Fruit Store'  # subject line give in assignment for email
    email_body = 'All fruits are uploaded to our website successfully. A detailed list is attached to this email.'  # body line give in assignment for email
    msg = generate_email("automation@example.com", "{}@example.com".format(user),
                         email_subject, email_body, "/tmp/processed.pdf")  # structuring email and attaching the file. Then sending the email, using the cus$
    send_email(msg)

### emails.py

In [ ]:
#!/usr/bin/env python3

import email
import mimetypes
import smtplib
import os


def generate_email(sender, recipient, subject, body, attachment_path):
    """Creates an email with an attachement."""
    # Basic Email formatting
    message = email.message.EmailMessage()
    message["From"] = sender
    message["To"] = recipient
    message["Subject"] = subject
    message.set_content(body)

    # Making attachment_path optional, if the attachment variable is empty string, no email will be sent.
    if not attachment_path == "":
        # Process the attachment and add it to the email
        attachment_filename = os.path.basename(attachment_path)
        mime_type, _ = mimetypes.guess_type(attachment_path)
        mime_type, mime_subtype = mime_type.split('/', 1)

        with open(attachment_path, 'rb') as ap:
            message.add_attachment(ap.read(), maintype=mime_type, subtype=mime_subtype,
                                   filename=attachment_filename)

    return message

def send_email(message):
    """Sends the message to the configured SMTP server."""
    mail_server = smtplib.SMTP('localhost')
    mail_server.send_message(message)
    mail_server.quit()


### health_check.py

In [ ]:
#!/usr/bin/env python3
import socket
import shutil
import psutil
import emails

def check_localhost():
    localhost = socket.gethostbyname('localhost')
    
    return localhost == "127.0.0.1"

def check_disk_usage(disk):
    """Verifies that there's enough free space on disk"""
    du = shutil.disk_usage(disk)
    free = du.free / du.total * 100
    
    return free > 20

def check_memory_usage():
    """Verifies that there's enough free space on disk"""
    mu = psutil.virtual_memory().available
    total = mu / (1024.0 ** 2)
    
    return total > 500

def check_cpu_usage():
    """Verifies that there's enough unused CPU"""
    usage = psutil.cpu_percent(1)
    
    return usage < 80

def send_email(subject):
    email = emails.generate_email("automation@example.com", "***********************@example.com",
                                  subject,
                                  "Please check your system and resolve the issue as soon as possible.", "")
    emails.send_email(email)

# If there's not enough disk, or not enough CPU, print an error
if not check_cpu_usage() :
    subject="Error - CPU usage is over 80%"
    
    print(subject)
    send_email(subject)
    
if not check_memory_usage():
    subject = "Error - Available memory is less than 500MB"
    
    print(subject)

if not check_disk_usage('/') :
    subject = "Error - Available disk space is less than 20%"
    
    print(subject)
    send_email(subject)

if not check_localhost():
    subject = "Error - localhost cannot be resolved to 127.0.0.1"
    
    print(subject)
    send_email(subject)
